# Processing

In [ ]:
!mkdir sparse_vector

In [ ]:
import pandas as pd
from tqdm import tqdm_notebook, tqdm
from collections import defaultdict
import numpy as np
from sparse_vector.sparse_vector import SparseVector
import os
from joblib import load, dump, Parallel, delayed

taget_antigen = ['DNase-seq', 'Histone', 'RNA polymerase', 'TFs and others']
chroms = [f'chr{i}' for i in list(range(1, 23)) + ['X', 'Y','M']]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def chrom_reader(chrom):
    files = sorted([i for i in os.listdir(f'/content/drive/My Drive/data_for_gans/data/hg19_dna/') if f"{chrom}_" in i])
    return ''.join([load(f"/content/drive/My Drive/data_for_gans/data/hg19_dna/{file}") for file in files])

# DNA sequence downloading and processing

In [ ]:
dna = {chrom:chrom_reader(chrom) for chrom in tqdm_notebook(chroms)}
lens_of_chroms = {chrom: len(dna[chrom]) for chrom in dna}
del dna

<ipython-input-8-0227fbfed577>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  dna = {chrom:chrom_reader(chrom) for chrom in tqdm_notebook(chroms)}


  0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
endoquad = pd.read_csv(f'/content/EndoQuad_hg19.bed', sep='\t', comment='t', header=None)

In [ ]:
endoquad

,0,1,2,3,4,5
0,chr1,10446,10467,Level1,0,-
1,chr1,10475,10497,Level1,0,-
2,chr1,17466,17501,Level3,0,-
3,chr1,17568,17589,Level1,0,-
4,chr1,136376,136404,Level1,0,+
...,...,...,...,...,...,...
391498,chrX,153595996,153596015,Level2,0,+
391499,chrX,153596172,153596204,Level4,0,-
391500,chrX,153596522,153596556,Level4,0,+
391501,chrX,153061880,153061919,Level1,0,-


In [ ]:
high_dataset = pd.concat([lvl4, lvl5, lvl6])
high_dataset = high_dataset.sort_values(by=["chrom"])

In [ ]:
high_dataset

,chrom,start,end
21,chr1,714494,714535
12145,chr1,22263821,22263851
12098,chr1,19986324,19986371
12060,chr1,19923498,19923536
12055,chr1,19812386,19812418
...,...,...,...
386811,chrX,118826820,118826847
391439,chrX,153570447,153570484
390509,chrY,21154559,21154601
390510,chrY,21154659,21154681


In [ ]:
columns = ["chrom", "start", "end"]
data = [zdna]
for d in data:
  d.columns = columns

In [ ]:
loc_dd = {chrm:SparseVector(lens_of_chroms[chrm]) for chrm in chroms}
df = df
for chrom, sub_df in tqdm(df.groupby('chrom')):
    if chrom not in chroms:
        continue
    vec = np.zeros(lens_of_chroms[chrom])
    for inter in sub_df.values:
        vec[inter[1]:inter[2]+1] = np.maximum(vec[inter[1]:inter[2]+1], 1)
    loc_dd[chrom] = SparseVector(vec)

dump(loc_dd, f'endoquad_high_levels_small.pkl', 3)